# Library

In [1494]:
import pandas as pd
import numpy as np
import os
import re
import collections
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Functions

In [1495]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [1496]:
# def filter_byindex(x,y):
#     #selectors = [x for x in col2]
#     return list(itertools.compress(x,y))

In [1497]:
def check_len(x,y):
    if x==y:
        return 0
    else:
        return 1

In [1498]:
def get_lf(i,j,k):
    if pd.isnull(k):
        regex_lf = re.compile(r'((?:\w+\W+){1,'+str(len(i))+'})\(\s'+i[0]+'.*\)')
        return regex_lf.findall(j)
    else:
        return k

In [1499]:
def ngram_filter(doc, word, n):
    tokens = doc.split()
    all_ngrams = ngrams(tokens, n)
    filtered_ngrams = [x for x in all_ngrams if word in x]
    return filtered_ngrams

In [1500]:
# def get_longform(tokens, acro, margin = 2, i =1):
#     long_form = ''
#     #Looking for before
#     for word in tokens[index-margin-len(acro):index]:
#         #if first letter of word is equal to first letter os acronym
#         if word[0] == acro[i].lower():
#             long_form += word + ' '
#             i += 1
#             if i == len(acro):
#                 break
#         elif (i == 1) and (word[0] == acro[i-1].lower()):
#             long_form = word + ' '
#             i = 1
#             if i == len(acro):
#                 break
#     long_form = long_form.rstrip()
#     return long_form
    

In [1501]:
def get_longform(tokens, acro, long):
    acro = acro.lower()
    long_form = ''
    margin = 2
    i =0
    #Looking for before
    if acro not in tokens:
        return -1
    if pd.isna(long):
        index = tokens.index(acro)
        for word in tokens[index-margin-len(acro):index]:
            #if first letter of word is equal to first letter os acronym
            if word[0] == acro[i].lower():
                long_form += word + ' '
                i += 1
                if i == len(acro):
                    break
            elif (i == 1) and (word[0] == acro[i-1].lower()):
                long_form = word + ' '
                i = 1
                if i == len(acro):
                    break
        long_form = long_form.rstrip()
        return long_form
    else:
        return long


# Load Data

### Trainning

318 clinical cases

In [1502]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/")

In [1503]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [1504]:
train_raw.head()

,doc_id,texto
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...


In [1505]:
train_raw.shape

(318, 2)

# Sub-track 1

## Found abbreviations (Short Forms)

### Regex

In [1506]:
patron3 = r'[A-Z]{2,8}' #Letras mayúsculas entre 2 y 8. Probar (\s|\()[A-Z]{2,8}
patron4 = r'\s[a-z]{1,2}\s' #Entre 2 y 3 letras minusculas entre espacios
patron5 = r'\b[aA-zZ]{1,4}\-[aA-zZ]{1,4}\b' #mayúsuclas o minúsculas entre guiones
#patron6 = r'[1-9]\s*[aA-zZ]{1,4}\/\b[aA-zZ]{1,4}\b' #Palabras divididas por / solo cuando las palabras no exceden de 4 caracteres
patron6 = r'\b\w{2}\b\/'
patron7 = r'\/\b\w{2}\b'
patron8 = r'[aA-zZ]{1,4}[A-Z]+[a-z]*[1-4]*'
#patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7, patron8))

In [1507]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

Add words to stopwords lists

In [1508]:
swords = swords + ['I','II','III','IV','V','VI','VII','VIII','IX', 'X', 'x']

### Get Short Formns with a regex in each text

In [1509]:
train_raw['abrev'] = train_raw['texto'].map(lambda x: generic_re.findall(x))

Separate mesurement units separate by "/"

In [1510]:
# train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [a.split("/") for a in x])

In [1511]:
# train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [item for sublist in x for item in sublist])

**Get offsets of the Short Forms founded in the text**

In [1512]:
train_raw['offse'] = train_raw['texto'].map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(generic_re, x)])

In [1513]:
#remove whitespaces
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i.strip() for i in x])

Filter Short Forms to delete ones which are stopwords, get their index too for filter offsets lists later

In [1514]:
# train_raw['abrev_index'] = train_raw['abrev'].apply(lambda x: [x.index(i) for i in x if i not in swords])
train_raw['abrev_index'] = train_raw['abrev'].apply(lambda x: [i for i,j in enumerate(x) if j not in swords])

In [1515]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i for i in x if i not in swords])

In [1516]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [a.replace('/',"") for a in x])

Filter offsets lists by index

In [1517]:
train_raw['offse'] = train_raw.apply(lambda x: [x['offse'][i] for i in x['abrev_index']], axis = 1)

In [1518]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]","[(789, 791), (1006, 1010)]","[24, 31]"
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[mm, mm, mg, mg, mg, mg, LDH, UI, GOT, UI, GPT...","[(1056, 1059), (1079, 1082), (1247, 1250), (12...","[36, 37, 44, 47, 48, 49, 50, 51, 52, 53, 54, 5..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[],[],[]
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg, dl, dl, AST, ALT, GGT, UI, dl, TAC, mg...","[(40, 43), (654, 657), (670, 673), (697, 700),...","[3, 23, 24, 25, 26, 27, 28, 29, 30, 37, 64, 65..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[],[],[]


In [1519]:
train_raw[train_raw['doc_id'] == 'S0004-06142005001000011-1']['abrev']

64    [IRC, IgA, Kg, Kg, EEII, EEII, ROT, RCP, RMN, ...
Name: abrev, dtype: object

Check abrev and offse columns has the same length

In [1520]:
train_raw['abrev_len'] =train_raw['abrev'].str.len()
train_raw['offse_len'] =train_raw['offse'].str.len()

In [1521]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]","[(789, 791), (1006, 1010)]","[24, 31]",2,2
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[mm, mm, mg, mg, mg, mg, LDH, UI, GOT, UI, GPT...","[(1056, 1059), (1079, 1082), (1247, 1250), (12...","[36, 37, 44, 47, 48, 49, 50, 51, 52, 53, 54, 5...",32,32
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[],[],[],0,0
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg, dl, dl, AST, ALT, GGT, UI, dl, TAC, mg...","[(40, 43), (654, 657), (670, 673), (697, 700),...","[3, 23, 24, 25, 26, 27, 28, 29, 30, 37, 64, 65...",22,22
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[],[],[],0,0


Check both lists have the same lenght

In [1522]:
train_raw['len_check'] = train_raw.apply(lambda row: check_len(row['abrev_len'],row['offse_len']),axis = 1)

Delete rows with different lenghts (check it later)

In [1523]:
train_raw.shape

(318, 8)

In [1524]:
train_raw[train_raw['len_check'] != 1].shape

(318, 8)

In [1525]:
#train_raw = train_raw[train_raw['len_check'] != 1]

### Text treatment

Remove string punctuation, lowecase, tokenize and remove stopwords

In [1526]:
for i in punctuation:
    train_raw['texto_clean'] = train_raw['texto'].str.replace(i,'').str.lower()

In [1527]:
train_raw['texto_clean'] = train_raw['texto_clean'].str.split().map(lambda x: ' '.join([w for w in x if w not in swords]))

In [1528]:
train_raw['tokens'] = train_raw['texto_clean'].map(lambda x: word_tokenize(x))

In [1529]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len,len_check,texto_clean,tokens
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]","[(789, 791), (1006, 1010)]","[24, 31]",2,2,0,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac..."
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[mm, mm, mg, mg, mg, mg, LDH, UI, GOT, UI, GPT...","[(1056, 1059), (1079, 1082), (1247, 1250), (12...","[36, 37, 44, 47, 48, 49, 50, 51, 52, 53, 54, 5...",32,32,0,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[],[],[],0,0,0,varón 33 años fumador paquete cigarrillos día ...,"[varón, 33, años, fumador, paquete, cigarrillo..."
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg, dl, dl, AST, ALT, GGT, UI, dl, TAC, mg...","[(40, 43), (654, 657), (670, 673), (697, 700),...","[3, 23, 24, 25, 26, 27, 28, 29, 30, 37, 64, 65...",22,22,0,"hombre 42 años, bebedor 100 g etanol día, ante...","[hombre, 42, años, ,, bebedor, 100, g, etanol,..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[],[],[],0,0,0,paciente 18 años edad 5 meses sido víctima gra...,"[paciente, 18, años, edad, 5, meses, sido, víc..."


### Get abreviations and offsets

Remove punctuation from abreviations

In [1530]:
# for i in punctuation:
#     train_raw['abrev'] = train_raw[train_raw['abrev'].notnull()]['abrev'].apply(lambda x: [a.replace(i,"") for a in x])

In [1531]:
train_raw['zip'] = train_raw.apply(lambda row: list(zip(row['abrev'], row['offse'])), axis = 1)

In [1532]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len,len_check,texto_clean,tokens,zip
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]","[(789, 791), (1006, 1010)]","[24, 31]",2,2,0,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...","[(RM, (789, 791)), (PAAF, (1006, 1010))]"
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[mm, mm, mg, mg, mg, mg, LDH, UI, GOT, UI, GPT...","[(1056, 1059), (1079, 1082), (1247, 1250), (12...","[36, 37, 44, 47, 48, 49, 50, 51, 52, 53, 54, 5...",32,32,0,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","[(mm, (1056, 1059)), (mm, (1079, 1082)), (mg, ..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[],[],[],0,0,0,varón 33 años fumador paquete cigarrillos día ...,"[varón, 33, años, fumador, paquete, cigarrillo...",[]
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg, dl, dl, AST, ALT, GGT, UI, dl, TAC, mg...","[(40, 43), (654, 657), (670, 673), (697, 700),...","[3, 23, 24, 25, 26, 27, 28, 29, 30, 37, 64, 65...",22,22,0,"hombre 42 años, bebedor 100 g etanol día, ante...","[hombre, 42, años, ,, bebedor, 100, g, etanol,...","[(g, (40, 43)), (mg, (654, 657)), (dl, (670, 6..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[],[],[],0,0,0,paciente 18 años edad 5 meses sido víctima gra...,"[paciente, 18, años, edad, 5, meses, sido, víc...",[]


### Get one row per abbreviation

Separate elements lists in different rows

In [1533]:
mine = train_raw.explode('zip')

In [1534]:
mine.shape

(3181, 11)

In [1535]:
mine[mine['zip'].isnull()].shape

(18, 11)

In [1536]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'zip']]

In [1537]:
mine = mine[mine['zip'].notnull()]

In [1538]:
mine.shape

(3163, 4)

In [1539]:
mine.head()

,doc_id,texto_clean,tokens,zip
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...","(RM, (789, 791))"
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...","(PAAF, (1006, 1010))"
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","(mm, (1056, 1059))"
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","(mm, (1079, 1082))"
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","(mg, (1247, 1250))"


### Separate SF from Offsets in different columns

In [1540]:
mine['abrev'] = mine.apply(lambda row: row['zip'][0], axis = 1)

In [1541]:
mine['offsets'] = mine.apply(lambda row: row['zip'][1], axis = 1)

In [1542]:
mine = mine[mine['abrev'] != ""]

In [1543]:
mine.head()

,doc_id,texto_clean,tokens,zip,abrev,offsets
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...","(RM, (789, 791))",RM,"(789, 791)"
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...","(PAAF, (1006, 1010))",PAAF,"(1006, 1010)"
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","(mm, (1056, 1059))",mm,"(1056, 1059)"
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","(mm, (1079, 1082))",mm,"(1079, 1082)"
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...","(mg, (1247, 1250))",mg,"(1247, 1250)"


Separate offsets tuples in different columns

In [1544]:
mine[['startOffset', 'endOffset']] = pd.DataFrame(mine['offsets'].tolist(), index=mine.index) 

In [1545]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'abrev', 'startOffset', 'endOffset']]

In [1546]:
mine.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",RM,789,791
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",PAAF,1006,1010
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1056,1059
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1079,1082
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mg,1247,1250


Delete null values, and change ttype to integer for offsets columns

In [1547]:
mine.shape

(3163, 6)

In [1548]:
mine.dropna(subset=['startOffset', 'endOffset'], inplace = True)

In [1549]:
mine.shape

(3163, 6)

In [1550]:
mine['startOffset'] = mine['startOffset'].astype(int)
mine['endOffset'] = mine['endOffset'].astype(int)

In [1551]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'abrev', 'startOffset', 'endOffset']]

In [1552]:
mine.shape

(3163, 6)

In [1553]:
#mine[mine.duplicated(subset=['doc_id', 'texto_clean', 'abrev', 'startOffset', 'endOffset'], keep=False)]

In [1554]:
mine = mine.drop_duplicates(subset = ['doc_id', 'texto_clean', 'abrev', 'startOffset', 'endOffset'])

In [1555]:
mine.shape

(3163, 6)

In [1556]:
mine.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",RM,789,791
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",PAAF,1006,1010
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1056,1059
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1079,1082
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mg,1247,1250


In [1557]:
# mine['abrev'] = mine['abrev'].apply(lambda x: [a.split("/") for a in x])

Check dataframes for one text

In [1558]:
# mine[mine['doc_id'] == 'S1130-05582012000300005-1']

In [1559]:
# train_abbr[train_abbr['doc_id'] == 'S1130-05582012000300005-1']

## Search Long Formns

### Search Long Forms in the same text

**measurement units dictionary**

In [1560]:
mu_dic = {"mL":"mililitro",
"mg":"miligramo",
"g":"gramo",
"L":"litro",
"mcg":"microgramo",
"mmol":"milimol",
"UI":"Unidades Internacionales",
"Miles UI":"Miles de Unidades Internacionales",
"Millones UI":"Millones de Unidades Internacionales",
"UFC":"Unidades Formadoras de Colonias",
"mEq":"miliequivalente",
"ng":"manogramo",
"Lf":"Unidad Floculante",
"UFP":"Unidad Formadora de Placa",
"DIC":"Dosis Infectante Mediana de Cultivo Celular 50% ",
"DIT":"Dosis Infectante Mediana de Cultivo Tisular 50% ",
"DI":"Dosis Infectante 50% ",
"mol":"Peso Molecular Gramo ",
"Eq":"Peso Equivalente Gramo ",
"Dosis":"Dosis",
"Almh":"Almohadilla",
"Amp":"Ampolla",
"Anl":"Anillo",
"Bar":"Barra",
"Bolsa":"Bolsa",
"Cap":"Capsula",
"Car":"Caramelo",
"Carp":"Carpula",
"Cart":"Cartucho",
"Com":"Comprimido",
"Dia":"Dia",
"Fras":"Frasco",
"Fras-Amp":"Frasco Ampolla ",
"Grag":"Gragea",
"Hora":"Hora",
"Imp":"Implante",
"Jab":"Jab¢n",
"Jer":"Jeringa Prellenada ",
"uL":"Microlitro",
"Ovu":"Ovulo",
"Parche":"Parche",
"Past":"Pastilla",
"Perl":"Perla",
"Pil":"Pildora",
"Pip":"Pipeta",
"%":"Porcentaje",
"Sach":"Sachet",
"Sob":"Sobre",
"Sup":"Supositorio",
"Tab":"Tableta",
"Troc":"Trocisco",
"Vial":"Vial",
"Kg":"Kilogramo",
"Gal":"Galon",
"Sis":"Sistema Terapeutico",
"mCi":"miliCuries",
"mBq":"milibequerel",
"UEL":"Unidades ELISA ",
"DL":"Dosis Letal",
"U USP":"Unidades USP ",
"U":"Unidades",
"Rot":"Rotacaps",
"CCID":"Dosis Infecciosa en Cultivo de Célula ",
"U":"UNIDAD",
"Otros":"Otros",
"µ Ci":"mcroCuries",
"Esp":"Esporas",
"mcHA":"microgramos de HA ",
"Gom":"Goma",
"KIU":"Unidad Inhibidora de Calicreina ",
"mcel":"Millones de Células ",
"DU":"Unidades de Antigeno D",
"Dil D2":"Dil D2",
"Tin.Mad.":"Tintura Madre",
"Dil D4":"Dil D4",
"Dil D5":"Dil D5",
"Dil D1":"Dil D1",
"Dil D8":"Dil D8",
"Dil D3":"Dil D3",
"OU":"Unidad de Opacidad ",
"mm": "milimetro",
"dm": "decimetro",
"cm": "centimetro"}

In [1561]:
mine['long_form'] = mine['abrev'].map(mu_dic)

In [1562]:
mine.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",RM,789,791,NaN
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",PAAF,1006,1010,NaN
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1056,1059,milimetro
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1079,1082,milimetro
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mg,1247,1250,miligramo


In [1563]:
# mine_lf = mine[mine['abrev'].str.len() > 1].reset_index()

### Get LF before SF

In [1564]:
# mine[mine['doc_id'] == 'S0004-06142005000900013-1'].iloc[0]['tokens']

With get_longform function, words befores SF that starts with SF letters will be detected

In [1565]:
def get_longform(tokens, acro, long):
    acro = acro.lower()
    long_form = ''
    margin = 2
    i =0
    #Looking for before
    if acro not in tokens:
        return -1
    if pd.isna(long):
        index = tokens.index(acro)
        for word in tokens[index-margin-len(acro):index]:
            #if first letter of word is equal to first letter os acronym
            if word[0] == acro[i].lower():
                long_form += word + ' '
                i += 1
                if i == len(acro):
                    break
            elif (i == 1) and (word[0] == acro[i-1].lower()):
                long_form = word + ' '
                i = 1
                if i == len(acro):
                    break
        long_form = long_form.rstrip()
        #print(re.split(' |-',long_form), len(re.split(' |-',long_form)), len(acro)-1)
        if len(re.split(' |-',long_form)) == (len(acro)):
            return long_form
    else:
        return long


In [1566]:
# mine[mine['doc_id'] == 'S1137-66272013000200023-1'].apply(lambda row: get_longform(row['tokens'], row['abrev'], row['long_form']), axis = 1)

In [1567]:
mine['long_form'] = mine.apply(lambda row: get_longform(row['tokens'], row['abrev'], row['long_form']), axis = 1)

In [1568]:
mine.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",RM,789,791,resonancia magnética
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",PAAF,1006,1010,None
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1056,1059,milimetro
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1079,1082,milimetro
1,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mg,1247,1250,miligramo


In [1569]:
mine.shape

(3163, 7)

### Number of SF and LF in tetxs

**Short Forms**

In [1570]:
mine['abrev'].nunique()

695

In [1571]:
pd.options.display.max_rows = 1000

In [1576]:
# mine['abrev'].value_counts().sort_index()

Delete rows with SF that are just numbers or number + letter

In [1573]:
mine[(mine['abrev'].str.len() <= 2) & (~mine['abrev'].str.isalpha())].head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
3,S0210-56912006000800008-1,"hombre 42 años, bebedor 100 g etanol día, ante...","[hombre, 42, años, ,, bebedor, 100, g, etanol,...",5U,2435,2438,-1
12,S1699-695X2015000100013-1,varón 22 años edad acude consulta atención pri...,"[varón, 22, años, edad, acude, consulta, atenc...",70,614,617,-1
18,S0212-16112009000600017-1,trata varón 60 años antecedentes personales ca...,"[trata, varón, 60, años, antecedentes, persona...",24,2298,2301,None
18,S0212-16112009000600017-1,trata varón 60 años antecedentes personales ca...,"[trata, varón, 60, años, antecedentes, persona...",24,3349,3352,None
23,S1134-80462015000100006-1,paciente femenina 49 años (peso: 60 kilos; tal...,"[paciente, femenina, 49, años, (, peso, :, 60,...",10,192,195,None


In [1574]:
mine.shape

(3163, 7)

In [1575]:
mine[~((mine['abrev'].str.len() <= 2) & (~mine['abrev'].str.isalpha()))].shape

(3059, 7)

In [1577]:
mine = mine[~((mine['abrev'].str.len() <= 2) & (~mine['abrev'].str.isalpha()))]

In [1579]:
# mine['abrev'].value_counts().sort_index()

**Long Forms**

In [1472]:
mine['long_form'].nunique()

77

### Study LF founded or not founded

Study SF without LF in the running text

In [1450]:
mine[mine['long_form'] == -1].shape

(921, 7)

Check how many texts don't have LF for the SF in the same text

In [1451]:
mine[(mine['abrev'].notnull()) & (mine['long_form']== '')]

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
14,S0212-71992006000800007-1,paciente mujer raza blanca 66 años edad antece...,"[paciente, mujer, raza, blanca, 66, años, edad...",f,882,885,
14,S0212-71992006000800007-1,paciente mujer raza blanca 66 años edad antece...,"[paciente, mujer, raza, blanca, 66, años, edad...",q,3841,3844,
23,S1134-80462015000100006-1,paciente femenina 49 años (peso: 60 kilos; tal...,"[paciente, femenina, 49, años, (, peso, :, 60,...",h,1018,1021,
27,S1130-05582014000400006-1,trata paciente masculino 52 años edad antecede...,"[trata, paciente, masculino, 52, años, edad, a...",h,539,542,
27,S1130-05582014000400006-1,trata paciente masculino 52 años edad antecede...,"[trata, paciente, masculino, 52, años, edad, a...",h,1377,1380,
27,S1130-05582014000400006-1,trata paciente masculino 52 años edad antecede...,"[trata, paciente, masculino, 52, años, edad, a...",d,1734,1737,
33,S0212-16112011000600041-1,"mujer 25 años, fumadora 10 cigarrillos/día, an...","[mujer, 25, años, ,, fumadora, 10, cigarrillos...",u,601,604,
44,S1137-66272015000300014-1,"mujer 66 años, intervenida histerectomía anexe...","[mujer, 66, años, ,, intervenida, histerectomí...",l,845,848,
47,S0378-48352005000400004-1,"varón 32 años, edad, acude urgencias hospital,...","[varón, 32, años, ,, edad, ,, acude, urgencias...",u,872,875,
58,S1134-80462015000200005-1,"presenta caso varón, 32 años (78 kg, 179 cm), ...","[presenta, caso, varón, ,, 32, años, (, 78, kg...",h,3080,3083,


In [1452]:
lf_null = mine[(mine['abrev'].notnull()) & (mine['long_form'] == '')].shape[0]
lf_null

42

In [1453]:
print(f"LF has not be found in the same text where the SF is in {lf_null/mine.shape[0]*100: .2f}% of texts")

LF has not be found in the same text where the SF is in  1.33% of texts


**Create dictionary with SF and LF pairs founded in the same text**

In [1454]:
df_pairs = mine[(mine['abrev'].notnull()) & (mine['long_form'] != '')]

In [1455]:
df_pairs.reset_index(drop = True, inplace = True)

In [1456]:
df_pairs.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",RM,789,791,resonancia magnética
1,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",PAAF,1006,1010,None
2,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1056,1059,milimetro
3,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mm,1079,1082,milimetro
4,S0212-71992005000400009-1,"trataba varón 27 años edad, sufrido neumonía s...","[trataba, varón, 27, años, edad, ,, sufrido, n...",mg,1247,1250,miligramo


In [1459]:
pairs_dic = {}
for index, row in df_pairs.iterrows():
    if (row['long_form'] != -1) & (row['long_form'] != None):
        if not row['abrev'] in pairs_dic:
            pairs_dic[row['abrev']] = set()
        pairs_dic[row['abrev']].add(row['long_form'])

In [1460]:
pairs_dic

{'RM': {'resonancia magnética'},
 'mm': {'milimetro'},
 'mg': {'miligramo'},
 'g': {'gramo'},
 'TAC': {'tomografia axial computarizada',
  'tomografía axial computadorizada',
  'tomografía axial computarizada',
  'tomografía axial computerizada'},
 'UCI': {'unidad cuidados intensivos'},
 'cm': {'centimetro'},
 'AP': {'atención primaria'},
 'PRC': {'programa rehabilitación cardiaca'},
 'UI': {'Unidades Internacionales'},
 'TC': {'tomografía computadorizada',
  'tomografía computarizada',
  'tomografía computerizada'},
 'RMN': {'resonancia magnética nuclear'},
 'LLA': {'leucemia linfoblástica aguda'},
 'FO': {'fondo ojo'},
 'CDI': {'cockcroft-gault diagnostica'},
 'TCS': {'tejido celular subcutáneo'},
 'mEq': {'miliequivalente'},
 'SPRL': {'servicio prevención riesgos laborales'},
 'OD': {'ojo derecho'},
 'OI': {'ojo izquierdo'},
 'RNM': {'resonancia nuclear magnética'},
 'PFP': {'pruebas función pulmonar'},
 'ECE': {'enteroscopia cápsula endoscópica'},
 'AV': {'agudeza visual'},
 'DPAR'

In [328]:
sorted(pairs_dic.keys())

['12',
 'A-P',
 'AA',
 'AAS',
 'AAT',
 'ACL',
 'AEO',
 'AFG',
 'AFP',
 'AGF',
 'AI',
 'AINES',
 'AL',
 'ALAT',
 'ALT',
 'AMA',
 'ANA',
 'ANCA',
 'ANCAS',
 'ANOES',
 'AO',
 'AP',
 'APC',
 'ASAT',
 'ASLO',
 'AST',
 'ATM',
 'AV',
 'AVI',
 'AVSC',
 'Alfa-feto',
 'BA',
 'BAL',
 'BAS',
 'BD',
 'BM',
 'BMC',
 'BMU',
 'BPA',
 'C-GSF',
 'C-Kit',
 'CA',
 'CD',
 'CDI',
 'CEA',
 'CK',
 'CMV',
 'COL',
 'CPK',
 'CT',
 'CU',
 'CX',
 'D-AAT',
 'DA',
 'DAI',
 'DAKO',
 'DFVA',
 'DG',
 'DII',
 'DM',
 'DP',
 'DPAR',
 'DR',
 'DRNS',
 'DTM',
 'EA',
 'EBHGA',
 'ECE',
 'ECG',
 'ECMO',
 'ECS',
 'EEII',
 'EF',
 'EFK',
 'EIA',
 'EID',
 'ELA',
 'EMA',
 'EMG',
 'ENA',
 'ENG',
 'EPOC',
 'EPR',
 'ESCHAP',
 'EVA',
 'FA',
 'FAV',
 'FID',
 'FMO',
 'FO',
 'FiO2',
 'GC',
 'GER',
 'GGT',
 'GOT',
 'GPT',
 'HBPM',
 'HCT',
 'HLA',
 'HPTS',
 'HPV',
 'HQR',
 'HSA',
 'HTA',
 'HTBI',
 'IAM',
 'ICG',
 'IF',
 'IGRA',
 'INSS',
 'IgA',
 'IgE',
 'IgG',
 'IgG4',
 'IgM',
 'LBA',
 'LCCT',
 'LCR',
 'LDH',
 'LH',
 'LLA',
 'LLBCP',
 'LLI',

In [1461]:
len(pairs_dic.keys())

68

## Check text individually

In [745]:
# texto = train_raw.iloc[4]['texto']
# texto

In [833]:
texto = df_pairs.iloc[467]['texto_clean']
texto

'varón 60 años edad acude consulta enero 2009 episodios hematuria macroscópica ocasional dos años evolución. transcurridos 10 meses primer episodio empezó polaquiuria disuria, 2 meses después realizó estudio ecográfico evidenciaba engrosamiento pared vesical derecha condicionaba ureterohidronefrosis. realizó estudio extensión mediante tac toraco-abdominal gammagrafía ósea apreciándose lesiones sugestivas metástasis confirmando hallazgos ecográficos. febrero 2009, recibió tratamiento quirúrgico mediante rtu vesical diagnóstico carcinoma plasmocitoide vejiga invasión capa muscular propia presencia focos embolización linfovascular, indicó cisto-prostatectomía radical derivación ileal practicó incidencias 27 marzo 2009. informe anatomopatológico reveló neoformación epitelial atípica crecimiento difuso nidos cordones, constituidos células pequeño tamaño, citoplasma escaso núcleo grande, pleomórfico polilobulado. además observaron áreas núcleos pleomórficos, polilobulados, grandes hipercromá

In [834]:
sf = generic_re.findall(texto)

In [835]:
sf_clean =  [i.strip() for i in sf if i.strip() not in swords]
# sf_clean

In [836]:
sf_clean = [a.replace('/',"") for a in sf_clean]

In [778]:
# for i in punctuation:
#     sf_clean = [a.replace(i,"") for a in sf_clean]

In [837]:
sf_clean

['rm', 'pet-tac', 'pet-tac', 'ng', 'j']

In [838]:
texto_clean = texto.split()

In [839]:
texto_clean = ' '.join([w for w in texto_clean if w not in swords])

In [840]:
for i in punctuation:
    texto_clean = texto_clean.replace(i,'').lower()

In [841]:
tokens = word_tokenize(texto_clean)
#tokens

In [847]:
def get_longform(tokens, acro):
    acro = acro.lower()
    long_form = ''
    margin = 2
    i =0
    #Looking for before
    if acro not in tokens:
        return -1
    index = tokens.index(acro)
    for word in tokens[index-margin-len(acro):index]:
        #if first letter of word is equal to first letter os acronym
        if word[0] == acro[i].lower():
            long_form += word + ' '
            i += 1
            if i == len(acro):
                break
        elif (i == 1) and (word[0] == acro[i-1].lower()):
            long_form = word + ' '
            i = 1
            if i == len(acro):
                break
    long_form = long_form.rstrip()
    if len(long_form) == len(acro):
        return long_form



In [848]:
for sf in sf_clean:
    a = get_longform(tokens, sf)
    print(f"Esta es la short form y su forma larga\n ({sf},{a})")

Esta es la short form y su forma larga
 (rm,None)
Esta es la short form y su forma larga
 (pet-tac,-1)
Esta es la short form y su forma larga
 (pet-tac,-1)
Esta es la short form y su forma larga
 (ng,-1)
Esta es la short form y su forma larga
 (j,None)


In [1061]:
mine[mine['abrev'] == 'RM']

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
0,S1130-05582012000300005-1,acude consultas paciente presenta tumoración c...,"[acude, consultas, paciente, presenta, tumorac...",RM,789,791,resonancia magnética
35,S1139-76322014000500014-1,niño cinco años derivado pediatra atención pri...,"[niño, cinco, años, derivado, pediatra, atenci...",RM,1202,1204,
65,S1137-66272013000200023-1,varón 60 años edad acude consulta enero 2009 e...,"[varón, 60, años, edad, acude, consulta, enero...",RM,2151,2153,realizó
75,S0376-78922016000200011-1,"mujer raza mestiza 43 años edad, antecedente m...","[mujer, raza, mestiza, 43, años, edad, ,, ante...",RM,1069,1071,resonancia magnética
75,S0376-78922016000200011-1,"mujer raza mestiza 43 años edad, antecedente m...","[mujer, raza, mestiza, 43, años, edad, ,, ante...",RM,1798,1800,resonancia magnética
94,S1134-80462009000500006-1,"presentamos caso paciente 55 años, remitido un...","[presentamos, caso, paciente, 55, años, ,, rem...",RM,1528,1530,resonancia magnética
101,S1698-44472005000400012-1,mujer 67 años edad consulta servicio tumefacci...,"[mujer, 67, años, edad, consulta, servicio, tu...",RM,697,699,
114,S0004-06142007000300012-1,paciente 52 años antecedentes personales insuf...,"[paciente, 52, años, antecedentes, personales,...",RM,2047,2049,renal
126,S0376-78922012000200008-1,mujer 46 años edad sometida mastectomía radica...,"[mujer, 46, años, edad, sometida, mastectomía,...",RM,2410,2412,radiólogo
144,S0210-48062009000400017-1,"varón 65 años, antecedente resección transuret...","[varón, 65, años, ,, antecedente, resección, t...",RM,1137,1139,resonancia magnética
